In [4]:
import numpy as np
import pandas as pd
pd.options.plotting.backend = 'plotly'
from my_stuff import MufexKeys
from quantfreedom.enums import *
from quantfreedom.exchanges.mufex_exchange.mufex import Mufex
from quantfreedom.indicators.indicators import qf_calc_rsi

from numba import njit
mufex_main = Mufex(
    api_key=MufexKeys.api_key,
    secret_key=MufexKeys.secret_key,
    use_test_net=False,
)

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
candles = mufex_main.get_candles(symbol="BTCUSDT", timeframe='5m', candles_to_dl=300)

It took 00 mins and 00 seconds to download 300 candles


In [9]:
prices = candles[:, 4]
period = 14

In [11]:
prices_shift = np.roll(prices, 1)
prices_shift[0] = np.nan

In [27]:
pchgs = (prices - prices_shift) / prices_shift

In [28]:
alpha = 1/ period
gains = np.where(pchgs > 0 , pchgs, 0)
losses = np.where(pchgs < 0 , -(pchgs), 0)

In [29]:
rma_gains = np.full_like(gains, np.nan)
rma_losses = np.full_like(losses, np.nan)

In [30]:
rma_gains[period] = gains[1 : period + 1].mean()
rma_losses[period] = losses[1 : period + 1].mean()

In [31]:
for i in range(period + 1, gains.size):
    rma_gains[i] = alpha * gains[i] + (1 - alpha) * rma_gains[i - 1]
    rma_losses[i] = alpha * losses[i] + (1 - alpha) * rma_losses[i - 1]

In [34]:
rs = rma_gains / rma_losses

In [36]:
rsi = 100 - (100 / (1 + rs))

In [37]:
rsi

array([        nan,         nan,         nan,         nan,         nan,
               nan,         nan,         nan,         nan,         nan,
               nan,         nan,         nan,         nan, 67.52596218,
       60.4847691 , 67.88097764, 64.93501992, 65.17073185, 63.58078993,
       63.71878761, 64.77713637, 64.95379186, 65.06988781, 61.60833861,
       53.56781763, 60.41881855, 61.3848905 , 65.22594861, 56.50263612,
       52.14262938, 57.04779274, 53.24712172, 51.34867122, 52.41335157,
       53.95053065, 57.07667699, 56.77369875, 57.64902912, 56.59872023,
       62.42378984, 64.92426352, 67.2547644 , 68.10826346, 67.13372421,
       61.62550618, 63.4456027 , 64.58955493, 59.00932921, 53.15157463,
       59.15401455, 62.59773631, 64.29490452, 61.64829384, 56.02326264,
       54.60610177, 56.65031939, 54.73374772, 48.62708976, 49.3341571 ,
       51.41145954, 56.68844464, 48.47637669, 49.62991291, 54.03695577,
       50.12562334, 51.43167243, 56.33224231, 55.72404745, 59.98